<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW3/blob/main/NLP_HW3_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import pandas as pd

from gensim.models.word2vec import Word2Vec
import gensim.downloader as gensim_api

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import LSTM, Dense, Input, Embedding, Bidirectional, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.models import Sequential

# Dataset

In [2]:
!gdown 11WeeMttH6I6MJ0t1h7FVSEtw0lKwpRA6
!gdown 1gLGNxjQzy6C8y4Oivr8etU1MMGfuKuEE
!gdown 127-sOeW6KMf6XNSAVM3bGjfwnmW0NciU

Downloading...
From: https://drive.google.com/uc?id=11WeeMttH6I6MJ0t1h7FVSEtw0lKwpRA6
To: /content/dev.conll
100% 478k/478k [00:00<00:00, 25.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gLGNxjQzy6C8y4Oivr8etU1MMGfuKuEE
To: /content/test.conll
100% 677k/677k [00:00<00:00, 35.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=127-sOeW6KMf6XNSAVM3bGjfwnmW0NciU
To: /content/train.conll
100% 11.4M/11.4M [00:00<00:00, 77.3MB/s]


In [3]:
def dependency_create(row):
  dependency_index = row["DependencyIndex"]
  index = row["Index"]

  if dependency_index == 0:
    return "Root"
  else:
    different = dependency_index - index

    if different > 0:
      return f"{different}R"
    elif different < 0:
      return f"{abs(different)}L"
    else:
      raise Exception(row)

In [4]:
def load_and_preprocess_dataset(path):
  dataset = pd.read_csv(path, delimiter="\t", header=None, names=["Index", "Word", "DependencyIndex"])
  dataset = dataset.dropna()
  dataset["Index"] = dataset["Index"].astype(int)
  dataset["DependencyIndex"] = dataset["DependencyIndex"].astype(int)
  dataset["Dependency"] = dataset.apply(dependency_create, axis=1)

  return dataset

In [5]:
train_dataset = load_and_preprocess_dataset("/content/train.conll")
valid_dataset = load_and_preprocess_dataset("/content/dev.conll")
test_dataset = load_and_preprocess_dataset("/content/test.conll")

In [6]:
label_dictionary = {"PAD": 0}

for i, label in enumerate(set(train_dataset["Dependency"].values.tolist() +
                              valid_dataset["Dependency"].values.tolist() +
                              test_dataset["Dependency"].values.tolist())):
  label_dictionary[label] = i+1

In [7]:
len(label_dictionary)

159

In [8]:
max(max(train_dataset["Index"].values), max(valid_dataset["Index"].values), max(test_dataset["Index"].values))

141

In [9]:
def convert_to_sequence(dataset):
  sequenced_dataset = []

  for i, item in dataset.iterrows():
    if item["Index"] == 1:
      if i>0:
        sequenced_dataset.append({"Words": new_words, "Labels": new_labels})
      new_words = [item["Word"]]
      new_labels = [item["Dependency"]]
    else:
      new_words.append(item["Word"])
      new_labels.append(item["Dependency"])
  
  sequenced_dataset.append({"Words": new_words, "Labels": new_labels})
  return pd.DataFrame(sequenced_dataset)

In [10]:
train_dataset = convert_to_sequence(train_dataset)
valid_dataset = convert_to_sequence(valid_dataset)
test_dataset = convert_to_sequence(test_dataset)

# Word2Vec

In [11]:
print(gensim_api.load("word2vec-google-news-300", return_path=True))

/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [12]:
word2vec_model = gensim_api.load("word2vec-google-news-300")

In [13]:
def apply_embedding(row):
  vectors = []
  for word in row["Words"]:
    try:
      vector = word2vec_model.get_vector(word)
    except KeyError:
      vector = np.zeros(300)
    vectors.append(vector)

  for i in range(141-len(row["Words"])):
    vectors.append(np.zeros(300))

  return np.stack(vectors)

In [14]:
def apply_label_index(row):
  labels = []
  for label in row["Labels"]:
    labels.append(label_dictionary[label])

  for i in range(141-len(row["Labels"])):
    labels.append(label_dictionary["PAD"])
  
  return np.stack(labels)

In [15]:
valid_dataset["Vector"] = valid_dataset.apply(apply_embedding, axis=1)
valid_dataset["LabelIndex"] = valid_dataset.apply(apply_label_index, axis=1)
test_dataset["Vector"] = test_dataset.apply(apply_embedding, axis=1)
test_dataset["LabelIndex"] = test_dataset.apply(apply_label_index, axis=1)

In [16]:
train_dataset = train_dataset.iloc[:1700]

train_dataset["Vector"] = train_dataset.apply(apply_embedding, axis=1)
train_dataset["LabelIndex"] = train_dataset.apply(apply_label_index, axis=1)

# BiLSTM Model

In [17]:
class BiLSTM(keras.Model):
  def __init__(self):
    super().__init__()
    self.model = Sequential([
                             Input((141, 300)),
                            #  word2vec_model.get_keras_embedding(),
                            #  Embedding(input_length=141 ,trainable=False),
                             Bidirectional(LSTM(32, return_sequences=True)),
                             TimeDistributed(Dense(159, activation="softmax"))
    ])
  
  def call(self, inputs):
    return self.model(inputs)

In [23]:
bilstm_model = BiLSTM()

In [24]:
bilstm_model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["Accuracy"])

In [25]:
es_callback = EarlyStopping(monitor="val_accuracy", patience=4, restore_best_weights=True)

In [26]:
bilstm_model.fit(x=np.stack(train_dataset["Vector"].values),
                 y=np.stack(train_dataset["LabelIndex"].values),
                 epochs=10,
                 validation_data=(np.stack(valid_dataset["Vector"].values),
                                  np.stack(valid_dataset["LabelIndex"].values)))

Epoch 1/10
54/54 [==============================] - 18s 254ms/step - loss: 3.1303 - Accuracy: 0.8244 - val_loss: 0.8404 - val_Accuracy: 0.8328
Epoch 2/10
54/54 [==============================] - 11s 207ms/step - loss: 0.6794 - Accuracy: 0.8520 - val_loss: 0.5693 - val_Accuracy: 0.8725
Epoch 3/10
54/54 [==============================] - 11s 204ms/step - loss: 0.5439 - Accuracy: 0.8720 - val_loss: 0.5028 - val_Accuracy: 0.8786
Epoch 4/10
54/54 [==============================] - 12s 216ms/step - loss: 0.5000 - Accuracy: 0.8746 - val_loss: 0.4742 - val_Accuracy: 0.8796
Epoch 5/10
54/54 [==============================] - 12s 215ms/step - loss: 0.4757 - Accuracy: 0.8755 - val_loss: 0.4554 - val_Accuracy: 0.8802
Epoch 6/10
54/54 [==============================] - 10s 193ms/step - loss: 0.4584 - Accuracy: 0.8763 - val_loss: 0.4405 - val_Accuracy: 0.8817
Epoch 7/10
54/54 [==============================] - 11s 203ms/step - loss: 0.4437 - Accuracy: 0.8786 - val_loss: 0.4277 - val_Accuracy: 0.8848